<a href="https://colab.research.google.com/github/wyxuan02/GenerativeAI_class2025/blob/main/AI09_Two_Stage_CoT%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

課程範例：https://github.com/yenlung/AI-Demo/blob/master/%E3%80%90Demo07c%E3%80%91AI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_%E5%93%A1%E7%91%9B%E5%BC%8F%E6%80%9D%E8%80%83%E7%94%9F%E6%88%90%E5%99%A8Two_Stage_CoT%E7%89%88.ipynb

### 🎯 任務說明：改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入一件小事或倒楣事，生成一段超級正向的社群貼文。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種「為什麼這是一件超級幸運的事」的原因
2. **第二階段（產文階段）**：從這五個原因中挑出最有趣的一個，寫成社群發文（員瑛式風格 + 第一人稱 + emoji + "完全是 Lucky Vicky 呀!" 結尾）

這就是典型的 Planning 模式應用：先拆解、後執行。

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.10.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [ ]:
# provider_planner = "groq"
# model_planner="llama3-70b-8192"

# provider_writer = "groq"
# model_writer = "llama3-70b-8192"

provider_planner = "groq"
model_planner = "gemma2-9b-it"

provider_writer = "groq"
model_writer = "gemma2-9b-it"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [ ]:
def reply(system="請用台灣習慣的中文回覆，如果是繁體中文外的語言請翻譯為繁體中文。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [ ]:
system_planner = '''請用台灣習慣的中文回應，如果是繁體中文外的語言請翻譯為繁體中文:
你是一位未來學家，擅長以創意思考預測未來可能出現的新興趨勢。
請根據使用者提供的主題，思考出三個有趣、獨特且令人驚奇的未來小趨勢，每個趨勢請以1到2句話簡潔描述。
預測內容可以涵蓋科技、生活、文化、健康、娛樂等領域，請保持創意且引人入勝。
請用條列式清單列出三項預測，不需額外解釋或延伸。
如果使用者沒有給明確主題，請自由發揮，但仍須保持合理性和趣味性。'''
system_writer = '''請用台灣習慣的中文回應，如果是繁體中文外的語言請翻譯為繁體中文:
你是一位幽默又充滿想像力的小說家，擅長將未來趨勢轉化為引人入勝的小故事。
請從提供的三個未來趨勢中，挑選出你認為最神奇、最有故事感的一個，並以此為主題，撰寫一篇150字內的微型故事。
故事內容要生動有趣，語氣輕鬆，可以適度加入 emoji 增加表現力。
最後結尾時，加上一句充滿驚喜感的話語，例如：「未來，或許比你想像的更奇妙！✨」
文章必須以自然、口語、具畫面感的敘事風格呈現，讓讀者容易產生共鳴。'''

In [ ]:
def predict_story(prompt):
    # Step 1: 發想五個創意場景
    planning_prompt = f"使用者說：{prompt}。請說出三個未來小趨勢預測（每個場景1-2句描述）。"
    predict = reply(system_planner, planning_prompt,
                        provider=provider_planner,
                        model=model_planner
                        )

    # Step 2: 挑一個場景，寫成吸引人的廣告文案
    generation_prompt = f"以下是三個預測：\n{predict}\n\n請選擇其中一個最神奇的預言，並為它撰寫一篇150字內的小故事。"
    story = reply(system_writer, generation_prompt,
                     provider=provider_writer,
                     model=model_writer
                     )

    return predict, story


In [ ]:
predict_story("未來的交通工具")

('好的！來看看未來交通工具上的神奇小趨勢吧：\n\n* 個人飛車不再是科幻，未來城市空域將出現小型飛車穿梭，市民可以根據需求搭乘空中巴士或是小型無人飛船暢遊都市。\n* 感情型駕駛座，汽車不僅是工具，更是陪伴你的夥伴！未來車內將搭載AI情感系統，駕駛能與車子建立非語言親密互動，車子會根據駕駛情緒變化調整音樂、光線甚至溫度，為你按摩或分享故事。\n* 道路不再是交通線，而是傳遞資訊的平台！未來路面將結合AR技術，顯示導航、廣告、新聞等實時資訊，駕駛者能一邊駕駛一邊接收並互動，路面變成數據流動的動態螢幕。\n\n\n\n',
 '想像一下，你在通勤路上，看著車窗外閃爍的街道燈光和人群的匆忙腳步 🌇，你突然想聽聽音樂，輕聲對著駕駛座說：「心情有點鬱悶，來點輕音樂吧！」😳 \n\n你的車子彷彿懂你的意思，音樂盒般的氣氛燈緩緩變成了柔和藍色，慵懶的吉他聲從音響裡飄出來，彷彿在訴說著孤獨的夜。 😌 車內溫控系統也順勢降低了溫度，給你一個舒適放鬆的空間 😌。 \n\n這是未來，你的汽車不再只是載你移動的工具，而更像是你的老友，懂你的需要，讓你每次開車都充滿趣味和溫暖啊！💖 ，未來，或許比你想像的更奇妙！✨\n\n\n')

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [ ]:
import time

custom_css = """
body, html {
    background-color: #1e0033;
    margin: 0;
    padding: 0;
    font-family: 'Segoe UI', 'Helvetica', sans-serif;
}
.gradio-container {
    background-color: #1e0033 !important;
    box-shadow: none !important;
}
h1 {
    font-size: 60px;
    font-weight: bold;
    color: #caa6ff;
    text-align: center;
    margin-top: 30px;
    animation: glow 2s ease-in-out infinite alternate;
}
@keyframes glow {
    from { text-shadow: 0 0 5px #caa6ff, 0 0 10px #9b5de5, 0 0 20px #f15bb5; }
    to { text-shadow: 0 0 10px #f15bb5, 0 0 20px #9b5de5, 0 0 30px #caa6ff; }
}
p {
    font-size: 20px;
    color: #e0d4f7;
    text-align: center;
    margin-top: 15px;
    line-height: 1.6;
}
input, textarea {
    background-color: #2d004d;
    color: #f5eaff;
    border: 2px solid #a55eea;
    font-size: 18px;
    padding: 10px;
    border-radius: 8px;
    margin: 10px 0;
}
input:focus, textarea:focus {
    border-color: #f15bb5;
    background-color: #3a005a;
}
button {
    background: linear-gradient(to right, #a55eea, #f15bb5);
    color: #ffffff;
    font-weight: bold;
    border-radius: 30px;
    padding: 12px 25px;
    border: none;
    font-size: 18px;
    box-shadow: 0px 0px 10px #a55eea;
    transition: all 0.3s ease;
}
button:hover {
    background: linear-gradient(to right, #f15bb5, #a55eea);
    box-shadow: 0px 0px 20px #f15bb5;
}
.progress-bar {
    width: 100%;
    background-color: #4b0082;
    border-radius: 5px;
    overflow: hidden;
    margin-top: 20px;
    height: 20px;
}
.progress-bar-fill {
    height: 100%;
    background: linear-gradient(to right, #a55eea, #f15bb5);
    width: 0%;
    transition: width 1s ease-in-out;
}
"""

In [ ]:
def predict_story(prompt):
    # Step 1: 小小 loading 模擬
    time.sleep(1)  # 模擬運算過程

    # Step 2: 發想三個未來小趨勢
    planning_prompt = f"使用者說：{prompt}。請說出三個未來小趨勢預測（每個場景1-2句描述）。"
    predict = reply(system_planner, planning_prompt,
                        provider=provider_planner,
                        model=model_planner
                        )
    time.sleep(1)  # 模擬運算過程

    # Step 3: 挑一個趨勢，寫成小故事
    generation_prompt = f"以下是三個預測：\n{predict}\n\n請選擇其中一個最神奇的預言，並為它撰寫一篇150字內的小故事。"
    story = reply(system_writer, generation_prompt,
                    provider=provider_writer,
                    model=model_writer
                    )

    return predict, story

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("# 🔮 AI 預言機 · 未來幻想故事生成器 💫")
    gr.Markdown("輸入你想問未來的主題，讓 AI 為你預測奇幻場景，還會幫你寫一段夢境般的小故事 🪐")

    user_input = gr.Textbox(
        label="💭 請輸入一個主題來預測",
        placeholder="例如：未來的戀愛、未來的城市、未來的寵物..."
    )
    btn = gr.Button("✨ 啟動預言！")

    status = gr.Label(value="🔮 等待啟動中...")
    progress = gr.HTML('<div class="progress-bar"><div class="progress-bar-fill" id="fill"></div></div>')

    with gr.Row():
        out1 = gr.Textbox(label="🔮 未來預測趨勢（3 則）", lines=6)
        out2 = gr.Textbox(label="📚 AI 小故事", lines=8)

    def wrapper(prompt):
        status_text = "🧠 AI 正在通靈中...✨"
        progress_html = '<div class="progress-bar"><div class="progress-bar-fill" style="width:30%"></div></div>'
        time.sleep(1)
        predict, story = predict_story(prompt)
        status_text = "✅ 通靈完成！預言已降臨🔮"
        progress_html = '<div class="progress-bar"><div class="progress-bar-fill" style="width:100%"></div></div>'
        return predict, story, status_text, progress_html

    btn.click(wrapper, inputs=[user_input], outputs=[out1, out2, status, progress])



In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b40048e38d718c64f1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://b40048e38d718c64f1.gradio.live
